In [1]:
import pandas as pd
import numpy as np
import glob
import os
import arrow

In [2]:
combined_df = pd.read_csv("./records_ant/rfid_reads_batch.csv", sep="|")

In [8]:
# Define the path to the folder containing the CSV files
folder_path = "./records_ant/"

# Use glob to get all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Initialize an empty list to hold the DataFrames
dfs = []

# Loop through the list of CSV files and read each one into a DataFrame
for file in csv_files:
    df = pd.read_csv(file, sep=";")
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# # Display the combined DataFrame
# print(combined_df)

In [15]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype                         
---  ------             --------------  -----                         
 0   Timestamp          800 non-null    object                        
 1   EPC                800 non-null    object                        
 2   Antenna            800 non-null    int64                         
 3   First Seen         800 non-null    int64                         
 4   LastSeen           800 non-null    int64                         
 5   Count              800 non-null    int64                         
 6   LastSeenTimestamp  800 non-null    datetime64[ns, America/Bogota]
dtypes: datetime64[ns, America/Bogota](1), int64(4), object(2)
memory usage: 43.9+ KB


In [10]:
# combined_df[" RSSI"] = combined_df[" RSSI"].str.replace(",", ".").astype(float)

In [26]:
combined_df.head()

,Timestamp,EPC,Antenna,First Seen,LastSeen,Count,LastSeenTimestamp,TimeGroup
0,2025-02-08 07:43:29.526,E280 1190 A503 006A 3554 F063,1,1739018606795153,1739018606795153,1,2025-02-08 07:43:26.795153-05:00,2025-02-08 07:43:20-05:00
1,2025-02-08 07:43:29.848,E280 1190 A503 006A 3554 1063,1,1739018606970372,1739018606970372,1,2025-02-08 07:43:26.970372-05:00,2025-02-08 07:43:20-05:00
2,2025-02-08 07:43:30.186,E280 1190 A503 006A 3554 1063,1,1739018607041444,1739018607041444,1,2025-02-08 07:43:27.041444-05:00,2025-02-08 07:43:20-05:00
3,2025-02-08 07:43:30.492,E280 1190 A503 006A 3554 F063,1,1739018607042223,1739018607042223,1,2025-02-08 07:43:27.042223-05:00,2025-02-08 07:43:20-05:00
4,2025-02-08 07:43:30.805,E280 1190 A503 006A 3554 F063,1,1739018607144289,1739018607144289,1,2025-02-08 07:43:27.144289-05:00,2025-02-08 07:43:20-05:00


In [25]:
# Agrupar por la columna 'Antenna' y calcular el promedio de 'LastSeenTimestamp'
combined_df['TimeGroup'] = (combined_df['LastSeenTimestamp'].dt.floor('20S'))
lap_counts = combined_df.groupby(['EPC', 'TimeGroup']).size().reset_index(name='LapCount')

# Mostrar el resultado
lap_counts

/tmp/ipykernel_18946/1999672589.py:2: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  combined_df['TimeGroup'] = (combined_df['LastSeenTimestamp'].dt.floor('20S'))


,EPC,TimeGroup,LapCount
0,E280 1190 A503 006A 3554 1063,2025-02-08 07:43:20-05:00,90
1,E280 1190 A503 006A 3554 1063,2025-02-08 07:58:40-05:00,10
2,E280 1190 A503 006A 3554 1063,2025-02-08 07:59:00-05:00,47
3,E280 1190 A503 006A 3554 1063,2025-02-08 08:08:20-05:00,64
4,E280 1190 A503 006A 3554 1063,2025-02-08 08:11:20-05:00,16
5,E280 1190 A503 006A 3554 1063,2025-02-08 08:11:40-05:00,175
6,E280 1190 A503 006A 3554 1063,2025-02-08 08:12:00-05:00,130
7,E280 1190 A503 006A 3554 F063,2025-02-08 07:43:20-05:00,80
8,E280 1190 A503 006A 3554 F063,2025-02-08 07:58:40-05:00,2
9,E280 1190 A503 006A 3554 F063,2025-02-08 07:59:00-05:00,31


In [ ]:
import pytz

# Obtener la lista de todas las zonas horarias
timezones = pytz.all_timezones

# Mostrar las primeras 10 zonas horarias como ejemplo
for tz in timezones:
    print(tz)

In [17]:
# combined_df["FirstSeenTimestamp"] = combined_df["First Seen"].apply(
#     lambda x: arrow.get(x).to("America/Bogota").format("YYYY-MM-DD HH:mm:ss.SSS")
# )

combined_df["LastSeenTimestamp"] = pd.to_datetime(
    combined_df["LastSeen"],unit="us", utc=True
).dt.tz_convert("America/Bogota")

In [92]:
import time

# epoch_time = 1613474400  # Replace with your epoch time
epoch_time = 1739018607144289  # Replace with your epoch time
print(time.gmtime(epoch_time))

formatted_time = time.strftime("%Y-%m-%d %H:%M:%S.%us", time.gmtime(epoch_time))
print(formatted_time)

time.struct_time(tm_year=55109275, tm_mon=11, tm_mday=25, tm_hour=6, tm_min=44, tm_sec=49, tm_wday=0, tm_yday=329, tm_isdst=0)
55109275-11-25 06:44:49.1s


In [ ]:
epoch_time = 1739018607144289  # Replace with your epoch time

# Convert epoch time to datetime
dt = arrow.get(epoch_time / 1e6)

# Localize to the desired timezone
localized_dt = dt.to("America/Bogota")

formatted_time = localized_dt.format("YYYY-MM-DD HH:mm:ss.SS.SSS")
print(formatted_time)

2025-02-08 07:43:27.14.144


In [ ]:
# importing datetime module
import datetime
import time

# assigned regular string date
date_time = datetime.datetime(2025, 2, 8, 7, 43, 30, 308050)

# print regular python date&time
print("date_time =>", date_time)

# displaying unix timestamp after conversion
print(
    "unix_timestamp => ",
    (time.mktime(date_time.timetuple()) + date_time.microsecond / 1e6),
)

date_time => 2025-02-08 07:43:30.308050
unix_timestamp =>  1739018610.30805


In [64]:
import time
import datetime

presentDate = datetime.datetime.now()
unix_timestamp = datetime.datetime.timestamp(presentDate) * 1000
print(unix_timestamp)

1740242160507.198


In [ ]:
time.gmtime(epoch_time)

time.struct_time(tm_year=55109275, tm_mon=11, tm_mday=24, tm_hour=2, tm_min=10, tm_sec=44, tm_wday=6, tm_yday=328, tm_isdst=0)